<a href="https://colab.research.google.com/github/chakra-ai/SentimentAnalysis/blob/master/NaiveBayesApproach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
url = 'https://raw.githubusercontent.com/chakra-ai/SentimentAnalysis/master/RomanUrduDataSet.csv'
df = pd.read_csv(url, encoding='utf8', header=None)
df.columns = ['text','target','junk']
df.drop('junk',axis=1, inplace=True)
df.dropna(inplace=True)
data = df[df['target'] != 'Neative']

# **Eliminate biased words are common across classes**

In [3]:
pos = df[df['target']=='Positive']['text'].values
neg = df[df['target']=='Negative']['text'].values
neu = df[df['target']=='Neutral']['text'].values

pos_corpus = []
neg_corpus = []
neu_corpus = []

for sentence in pos:
    words = sentence.split()
    pos_corpus = pos_corpus + words
pos_corpus = set(pos_corpus)

for sentence in neg:
    words = sentence.split()
    neg_corpus = neg_corpus + words
neg_corpus = set(neg_corpus)

for sentence in neu:
    words = sentence.split()
    neu_corpus = neu_corpus + words
neu_corpus = set(neu_corpus)

In [4]:
common_words = pos_corpus.intersection(neg_corpus, neu_corpus)
print(len(common_words))
print(common_words)

5387
{'agay', 'Pehli', 'kyoon', 'kaam', 'Ke', 'bana', 'Par', 'Jata', 'wicket', '40', 'Siyasi', 'Chemistry', 'maqbool', 'pizza', 'our', 'tahafuz', 'hmri', 'Bhar', 'mamlat', 'yaad', 'tery', 'baghair', 'nikl', 'Sa', 'aisay', 'Khanay', 'yar', 'Mery', 'sahab', 'haal', 'rya', 'hoi', 'idare', 'mary', 'dastoor', 'Call', 'Murad', 'waldain', 'copy', 'hii', '2013', 'Zakhmi', 'stadiums', 'Men', 'Logon', 'hoon', 'mene', 'm', 'warna', 'len', 'weight', 'Ayoub', '😂😂😂😂😂😂', 'Inter', 'Hon', 'manaya', 'Amna', 'Kitna', 'barae', 'sahir', 'aankh', 'sarhad', 'samait', 'December', 'dee', 'mustaqil', 'nara', 'always', 'Usi', 'Aafia', 'qism', '2006', 'girls', 'tk', 'milti', 'Still', 'halka', 'Sab', 'unhon', 'magar', 'imdad', 'me..', 'lgy', 'siasi', 'pakistani', 'Kash', 'baten', 'chale', 'Indian', 'Chauhanne', 'hona', 'Thik', 'pride', 'chaheye', "It's", 'Mushtaq', 'Multan', 'hon,', 'smile', 'hamin', 'Ravi', 'Rahi', 'hukam', 'alawa', '2009', 'Jaan', 'rahin', 'qaiym', 'choice', 'numayan', 'chalta', 'association', '

In [5]:
def clean_doc(doc):
    tokens = doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    tokens = [re_punc.sub('',w) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if not word in common_words]
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if len(word)>2]
    return tokens

In [6]:
#Get X and Y data
corpus = []
for i in range(data.shape[0]):
    review = data.iloc[:,0].values[i]
    review = clean_doc(review)
    review=' '.join(review)
    corpus.append(review)

X = np.array(corpus)

y = data.iloc[:,1].values

In [7]:
c_data = pd.DataFrame(X, columns=['text'])
c_data['label'] = y
c_data['char_len'] = c_data['text'].apply(lambda x: len(x) - x.count(" "))
c_data['word_count'] = c_data['text'].apply(lambda x: len(x.split(" ")))
c_data.shape

(20227, 4)

In [8]:
c_data.head(10)

,text,label,char_len,word_count
0,sai,Positive,3,1
1,,Positive,0,1
2,,Positive,0,1
3,,Positive,0,1
4,are wha,Positive,6,2
5,,Positive,0,1
6,wha khubiya,Positive,10,2
7,khubiya,Positive,7,1
8,,Positive,0,1
9,everyone allah swt hamesha lawzo zaat sub hey,Positive,38,8


In [9]:
c_data = c_data[c_data['char_len'] > 0]
print(c_data.shape)
c_data.head(10)

(17286, 4)


,text,label,char_len,word_count
0,sai,Positive,3,1
4,are wha,Positive,6,2
6,wha khubiya,Positive,10,2
7,khubiya,Positive,7,1
9,everyone allah swt hamesha lawzo zaat sub hey,Positive,38,8
10,rab kreem reham watneaziz frmaamin,Positive,30,5
11,jaago coom jaago nokar humarey tex tankha paki...,Positive,54,10
12,sacha jany kese bdl sakta think nsl ase guzr jain,Positive,40,10
13,garip lati zulum lati,Positive,18,4
14,insaaf,Positive,6,1


In [10]:
c_data.label.value_counts()

Neutral     7090
Positive    5313
Negative    4883
Name: label, dtype: int64

# Cleaned Data with 3 Classes 'Positive', 'Negative', 'Neutral'

In [11]:
c_data_3 = c_data.copy()

In [12]:
vocab_3 = []
for sentence in c_data_3['text'].values:
  temp = sentence.split()
  vocab_3 = vocab_3 + temp
print('Total Vacab Size in the 3 Classes dataset is : {}'.format(len(set(vocab_3))))

Total Vacab Size in the 3 Classes dataset is : 30166


# Cleaned Data with 2 Classes 'Positive', 'Negative'

In [13]:
c_data_2 = c_data[c_data['label'].isin(['Positive', 'Negative'])]
c_data_2.label.value_counts()

Positive    5313
Negative    4883
Name: label, dtype: int64

In [14]:
vocab_2 = []
for sentence in c_data_2['text'].values:
  temp = sentence.split()
  vocab_2 = vocab_2 + temp
print('Total Vacab Size in the 3 Classes dataset is : {}'.format(len(set(vocab_2))))

Total Vacab Size in the 3 Classes dataset is : 22521


# Prepare the Data

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [16]:
y_3 = c_data_3['label'].values
enc = LabelEncoder()
y_3 = enc.fit_transform(y_3)

In [17]:
y_3.shape

(17286,)

In [18]:
c_data_3[['text', 'char_len', 'word_count']].shape

(17286, 3)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(c_data_3[['text', 'char_len', 'word_count']],
                                                    y_3, 
                                                    test_size = 0.2,
                                                    random_state = 42,
                                                    stratify = y_3)

In [20]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2)) 
vectorizer = vectorizer.fit(X_train['text'])
x_train_v = vectorizer.transform(X_train['text'])
x_test_v = vectorizer.transform(X_test['text'])

In [ ]:
X_train_v_3 = pd.concat([X_train[['char_len', 'word_count']].reset_index(drop=True),
                          pd.DataFrame(x_train_v.toarray())], axis=1)
X_test_v_3 = pd.concat([X_test[['char_len', 'word_count']].reset_index(drop=True),
                         pd.DataFrame(x_test_v.toarray())], axis=1)

In [ ]:
# Shape of the Training data X and Y
print("Shape of the Training data X : {}, Y : {}".format(X_train_v_3.shape, y_train.shape))

In [ ]:
# n_jobs = -1 for building parallel 150 decision trees. 
# Max_depth = None means it will build decision tree until minminzation of loss
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1,class_weight="balanced",random_state=42)
rf_model = rf.fit(X_train_v_3, y_train)
y_pred = rf_model.predict(X_test_v_3)

precision = precision_score(y_test, y_pred, pos_label='positive', average='weighted')
recall = recall_score(y_test, y_pred, pos_label='positive', average='weighted')
accuracy = accuracy_score(y_test, y_pred)

print('Precision: {:.2f} | Recall: {:.2f} | Accuracy: {:.2f}'.format(precision,recall,accuracy))

sns.set(font_scale=1.4)#for label size
sns.heatmap(confusion_matrix(y_test, y_pred), cmap="Blues", annot=True,annot_kws={"size": 14}, fmt='g')# font size